to solve this question https://stackoverflow.com/questions/24612626/b-spline-interpolation-with-python/24693358#24693358

In [1]:
import os

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [2]:
import numpy as np
import scipy.interpolate as si

b spline interpolation from https://stackoverflow.com/questions/24612626/b-spline-interpolation-with-python

In [6]:
#curve base
points = [[-6, 2], [-3, -2],[0, 0], [0, 2], [2, 3], [4, 0], [6, 3], [8, 5], [8, 8], [6, 8], [5, 9], [7, 2]];
points = np.array(points)
x = points[:,0]
y = points[:,1]


t = range(len(points))
ipl_t = np.linspace(0.0, len(points) - 1, 100)


x_tup = si.splrep(t, x, k=3)
y_tup = si.splrep(t, y, k=3)

x_list = list(x_tup)
xl = x.tolist()
x_list[1] = xl + [0.0, 0.0, 0.0, 0.0]

y_list = list(y_tup)
yl = y.tolist()
y_list[1] = yl + [0.0, 0.0, 0.0, 0.0]

x_i = si.splev(ipl_t, x_list)
y_i = si.splev(ipl_t, y_list)

In [7]:
url = "https://cgm.demo-de.cloud.c3.cgm.ag/cgmg3"
service = Service(executable_path='./chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)

In [10]:
action =  ActionChains(driver);
startElement = driver.find_element(By.ID, '287e41c0-a5de-4102-8d17-5c247b015e9b')
#First, go to your start point or Element
action.move_to_element(startElement);
action.perform();

for mouse_x, mouse_y in zip(x_i, y_i):
    action.move_by_offset(mouse_x,mouse_y);
    action.perform();
    sleep(0.1)
    print(mouse_x, mouse_y)

-6.0 2.0
-5.509316415180612 1.3817634506934917
-5.037494284407864 0.856576741350404
-4.5848765432098775 0.4187242798353913
-4.151806127114769 0.062490474013107894
-3.7386259716506633 -0.2178402682517911
-3.3456790123456797 -0.42798353909465014
-2.973308184727938 -0.5736549306508153
-2.62185642432556 -0.6605700350556316
-2.2916666666666665 -0.6944444444444443
-1.983081847279378 -0.6809937509525987
-1.696444901691816 -0.6259335467154399
-1.432098765432099 -0.5349794238683128
-1.1903863740283493 -0.41384697454656305
-0.9716506630086881 -0.268251790885536
-0.7762345679012347 -0.10390946502057635
-0.6044810242341108 0.07346441091297046
-0.45673296753543674 0.2581542447797592
-0.3333333333333333 0.4444444444444444
-0.23365340649291264 0.6275339125133363
-0.15317786922725188 0.8062795305593662
-0.08641975308641991 0.9804526748971188
-0.0278920896204848 1.1498247218411823
0.027892089620484582 1.3141670477061422
0.08641975308641968 1.4732510288065843
0.15317786922725188 1.6268480414570947
0.233